In [43]:
import os
import json
from pathlib import Path

scn_files: list[Path] = []

folder_path = Path("./game_res/ks.scn.decompile")
for item in folder_path.iterdir():
    if item.is_file(): 
        scn_files.append(item.absolute())

print(f"{len(scn_files)} ks scn files found")

34 ks scn files found


In [44]:
def analyze_ks_scn(file_path) -> list:
    with open(file_path, "r", encoding="utf-8") as f:
        ks = json.loads(f.read())
    
    scene_list = []

    for scene in ks["scenes"]:
        text_list = []
        dialog_list = []
        if "texts" in scene:
            text_list = scene["texts"]
        for text in text_list:
            ja_name = text[0]
            ja_display_name = text[1]
            dialogs = text[2]
            if not dialogs: continue
            if text[3]:
                voice = [
                    {
                        "name": item["name"],
                        "source": item["voice"]
                    } for item in text[3]
                ]
            else: voice = None
            dialog_list.append({
                "name": [
                    ja_display_name if ja_display_name else ja_name,
                    dialogs[1][0],
                    dialogs[2][0],
                    dialogs[3][0]
                ],
                "text": [
                    dialogs[0][1],
                    dialogs[1][1],
                    dialogs[2][1],
                    dialogs[3][1]
                ],
                "voice": voice
            })
        nexts = None
        if "nexts" in scene:
            if len(scene["nexts"]) > 0:
                nexts = [
                    next["storage"]
                    for next in scene["nexts"]
                ]
        scene_list.append({
            "title": scene["title"],
            "next": nexts,
            "dialogs": dialog_list
        })
    
    return scene_list

解析后的每个文件表示一个游戏章节，每个章节包含若干场景，每个场景的结构如下：

```ts
{
    "name": string,                 // 章节文件名称
    "title": string[],              // 章节标题
    "next": string | null,          // 后续章节文件名称（可能多个）
    "dialogs": {                    // 对话列表
        "name": (string | null)[],  // 角色展示名称
        "text": string[],           // 对话内容
        "voice": null |{            // 角色语音
            "name": string,         // 角色实际名称（日语）
            "source": string        // 语音文件名称
        }
    }
}

```

In [45]:
os.makedirs("./game_res/ks.scn.chapters", exist_ok=True)
for file in scn_files:
    result = analyze_ks_scn(file)
    with open(f"./game_res/ks.scn.chapters/{file.stem}.json", "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=4)